# 🩻 Bone Fracture Detection using Deep Learning  

<img src="D:\python for data science\skeleton xray GIF.gif" width="400" />




In [ ]:
import tensorflow as tf 
from tensorflow import keras
from keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
traingen=ImageDataGenerator(    
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

train_genertor=traingen.flow_from_directory(
   r"D:\python for data science\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\train",
    target_size=(150 , 150),
    batch_size=32,
    class_mode="binary",
    color_mode="rgb"
)
val_genertor=traingen.flow_from_directory(
    r'\python for data science\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\val',
    target_size=(150 , 150),
    batch_size=32,
    class_mode="binary", 
    color_mode="rgb"
)
test_genertor=traingen.flow_from_directory(
    r'\python for data science\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\test',
    target_size=(150 , 150),
    batch_size=32,
    class_mode="binary" ,#علي حسب الداتا باينري كلاسفكيش ولا مالتي
    color_mode="rgb"
)


##Show some Images

In [ ]:
import cv2

img = train_genertor[0][0][0]

img = img.astype("uint8")
cv2.imshow("X-ray Sample", img)
cv2.waitKey(0)          
cv2.destroyAllWindows()  



In [ ]:
img = train_genertor[0][0][25]

img = img.astype("uint8")
cv2.imshow("X-ray Sample", img)
cv2.waitKey(0)          
cv2.destroyAllWindows()  


In [ ]:
img = train_genertor[0][0][15]

img = img.astype("uint8")
cv2.imshow("X-ray Sample", img)
cv2.waitKey(0)          
cv2.destroyAllWindows()  


In [ ]:
img = train_genertor[0][0][23]

img = img.astype("uint8")
cv2.imshow("X-ray Sample", img)
cv2.waitKey(0)          
cv2.destroyAllWindows()  


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = list(train_genertor.class_indices.keys())
counts = np.bincount(train_genertor.classes)

plt.bar(labels, counts)
plt.title("Class Distribution")
plt.show()


In [ ]:


model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
patience=3)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

hist=model.fit(train_genertor , epochs=10 , callbacks=[callback], validation_data=val_genertor)



In [ ]:
plt.plot(hist.history['loss'], label='train_loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss over epochs'); plt.xlabel('epoch'); plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label='train_acc')
plt.plot(hist.history['val_accuracy'], label='val_acc')
plt.legend(); plt.title('Accuracy over epochs'); plt.xlabel('epoch'); plt.show()

In [ ]:
loss, acc = model.evaluate(test_genertor)
print(f"Test Accuracy: {acc:.2f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

y_pred = (model.predict(test_genertor) > 0.5).astype("int32")
print(classification_report(test_genertor.classes, y_pred))

cm = confusion_matrix(test_genertor.classes, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.show()


#For Save The Model

In [ ]:
model.save("bone_fracture.h5")